# Research Paper Query bot


In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper,DuckDuckGoSearchAPIWrapper
# from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
# from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
from langchain_community.tools.asknews import AskNewsSearch

c:\ProgramData\anaconda3\envs\FreeGenAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os 
from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\Users\dewan\Coding\GenAIKN\Langchain\.env")

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
os.environ["ASKNEWS_CLIENT_SECRET"]=os.getenv("ASKNEWS_CLIENT_ID")
os.environ["ASKNEWS_API_KEY"]=os.getenv("ASKNEWS_API_KEY")

In [6]:
llm=ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)


## Tools

In [7]:
from langchain.tools import tool
from langchain.agents import create_agent


search = DuckDuckGoSearchResults(num_results=10)
@tool
def search(query:str)->str:
    """Search for information"""
    results = search.invoke(query)
    return results


# news = AskNewsSearch(max_results=2)
# @tool
# def news(query:str , hours_back:int = 24)->str:
#     """For Daily News"""
#     results = news.invoke({"query":query,"hours_back":hours_back})
#     return results

In [8]:
## Handle tool errors
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_error(request,handler):
    """Handle tool execution errors with custom messages"""
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=str(e),
            tool_call_id=request.tool_call["id"]
            )

In [16]:
tools = [search]


agent=create_agent(
    model=llm,
    tools=tools,
    middleware=[handle_tool_error],
    system_prompt="You are a agent to help blind people"
)

In [21]:
response = agent.invoke(
    {"messages":[{"role":"user","content":"tell me about Todays News"}]

    }
    )

: 

In [18]:
response

{'messages': [AIMessage(content='How can I assist you today? If you need information or resources, just let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 45, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Ca4sFUyKValOOmEOVufxANKv4HNuy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c34c3ac4-12e7-4424-a27d-ae190798ac1b-0', usage_metadata={'input_tokens': 45, 'output_tokens': 20, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}